In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 1. 加载模型

### 1. chat模型

In [1]:
from langchain_deepseek.chat_models import ChatDeepSeek  # 导入 DeepSeek 的聊天模型

deepseek_api_key = "sk-fffbb9b8a78d436a91a4780356b67a93"

llm = ChatDeepSeek(model="deepseek-chat", api_key = deepseek_api_key)

# response = llm.invoke(["你是一名咨询助理，请为华为写一个简介"])
# print(response.content)

### 2. embedding模型

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

### 3. 向量存储库

In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)